In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [ ]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row['mid_l'] > row.EMA_100 and row.candle_dir == BUY:
        return BUY
    if row.direction == SELL and row['mid_h'] < row.EMA_100 and row.candle_dir == SELL:
        return SELL
    return NONE

In [ ]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [ ]:
df_an = MACD(df_an)
df_an['EMA_100']=df_an['mid_c'].ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [ ]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0 :
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0 :
        return SELL
    return NONE

In [ ]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

candle_dir = df_an['mid_c'] - df_an['mid_o']
df_an['candle_dir'] = [BUY if x >= 0 else SELL for x in candle_dir]

In [ ]:
df_an.direction.value_counts()

In [ ]:
df_an.head()

In [ ]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an=MACD(df_an)
    df_an['EMA_100']=df_an['mid_c'].ewm(span=100, min_periods=100).mean()

    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    candle_dir = df_an['mid_c'] - df_an['mid_o']
    df_an['candle_dir'] = [1 if x >= 0 else -1 for x in candle_dir]
    
    our_cols=['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
              'bid_o', 'bid_h','bid_l', 'bid_c', 
              'ask_o', 'ask_h', 'ask_l', 'ask_c',
              'direction', 'candle_dir', 'EMA_100']
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )

    gt.run_test()
    return gt.df_results

In [ ]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD"]:
    res.append(dict(pair=p, res=run_pair(p, time_d=1)))

In [ ]:
for r in res:
    print(r['pair'], r['res'].result.sum())

In [ ]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD"]:
    res.append(dict(pair=p, res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())